In [1]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.7 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 9.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import re
import pickle
import string
import unicodedata
from random import randint

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from wordcloud import STOPWORDS, WordCloud

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, TimeDistributed

E0000 00:00:1753682142.921151      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0728 05:55:42.929509772      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0728 05:55:42.929524231      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0728 05:55:42.929527425      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0728 05:55:42.929529783      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0728 05:55:42.929532216      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

In [4]:
!pip install contractions==0.1.73

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 14.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
from contractions import contractions_dict

for key, value in list(contractions_dict.items())[:10]:
    print(f'{key} == {value}')

I'm == I am
I'm'a == I am about to
I'm'o == I am going to
I've == I have
I'll == I will
I'll've == I will have
I'd == I would
I'd've == I would have
Whatcha == What are you
amn't == am not


In [6]:
# Using TPU

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1753682158.410570      13 service.cc:145] XLA service 0x57a7fd4db760 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753682158.410624      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1753682158.410628      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1753682158.410631      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1753682158.410640      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1753682158.410643      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1753682158.410645      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1753682158.410648      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1753682158.410650      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:6, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


Loading data

In [7]:
train_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv")

In [8]:
train_data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
287108,fffdfb56fdf1a12d364562cc2b9b1d4de7481dee,By . James Rush . Former first daughter Chelse...,Chelsea Clinton said question of running for o...
287109,fffeecb8690b85de8c3faed80adbc7a978f9ae2a,An apologetic Vanilla Ice has given his first ...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,ffff5231e4c71544bc6c97015cdb16c60e42b3f4,America's most lethal sniper claimed he wished...,America's most lethal sniper made comment in i...
287111,ffff924b14a8d82058b6c1c5368ff1113c1632af,"By . Sara Malm . PUBLISHED: . 12:19 EST, 8 Mar...",A swarm of more than one million has crossed b...


In [9]:
test_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/test.csv")

In [10]:
test_data

,id,article,highlights
0,92c514c913c0bdfe25341af9fd72b29db544099b,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...
1,2003841c7dc0e7c5b1a248f9cd536d727f27a45a,A drunk teenage boy had to be rescued by secur...,Drunk teenage boy climbed into lion enclosure ...
2,91b7d2311527f5c2b63a65ca98d21d9c92485149,Dougie Freedman is on the verge of agreeing a ...,Nottingham Forest are close to extending Dougi...
3,caabf9cbdf96eb1410295a673e953d304391bfbb,Liverpool target Neto is also wanted by PSG an...,Fiorentina goalkeeper Neto has been linked wit...
4,3da746a7d9afcaa659088c8366ef6347fe6b53ea,Bruce Jenner will break his silence in a two-h...,"Tell-all interview with the reality TV star, 6..."
...,...,...,...
11485,ed8674cc15b29a87d8df8de1efee353d71122272,Our young Earth may have collided with a body ...,Oxford scientists say a Mercury-like body stru...
11486,2f58d1a99e9c47914e4b1c31613e3a041cd9011e,A man facing trial for helping his former love...,Man accused of helping former lover kill woman...
11487,411f6d57825161c3a037b4742baccd6cd227c0c3,A dozen or more metal implements are arranged ...,Marianne Power tried the tuning fork facial at...
11488,b5683ef8342056b17b068e0d59bdbe87e3fe44ea,Brook Lopez dominated twin brother Robin with ...,Brooklyn Nets beat the Portland Trail Blazers ...


In [11]:
val_data = pd.read_csv("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/validation.csv")

In [12]:
val_data

,id,article,highlights
0,61df4979ac5fcc2b71be46ed6fe5a46ce7f071c3,"Sally Forrest, an actress-dancer who graced th...","Sally Forrest, an actress-dancer who graced th..."
1,21c0bd69b7e7df285c3d1b1cf56d4da925980a68,A middle-school teacher in China has inked hun...,Works include pictures of Presidential Palace ...
2,56f340189cd128194b2e7cb8c26bb900e3a848b4,A man convicted of killing the father and sist...,"Iftekhar Murtaza, 29, was convicted a year ago..."
3,00a665151b89a53e5a08a389df8334f4106494c2,Avid rugby fan Prince Harry could barely watch...,Prince Harry in attendance for England's crunc...
4,9f6fbd3c497c4d28879bebebea220884f03eb41a,A Triple M Radio producer has been inundated w...,Nick Slater's colleagues uploaded a picture to...
...,...,...,...
13363,e93f721ba4949f21f33549c4a21d55ff456af979,All shops will be allowed to offer ‘click and ...,Shops won't have to apply for planning permiss...
13364,8df19a570ad14119a7d00f3bbe864fedf8c1691d,Mo Farah has had his nationality called into q...,Mo Farah broke the European half-marathon reco...
13365,2fdd5f89aa26e91ceea9b0ef264abfcfc3e6fa2e,Wolves kept their promotion hopes alive with a...,Wolves are three points off the play-off place...
13366,530d7b18d7a715b368b0745f9dfebfe353adeda8,A Brown University graduate student has died ...,"Hyoun Ju Sohn, a 25-year-old doctoral student,..."


In [13]:
def expand_contractions(text, contraction_map=contractions_dict):
    # Using regex for getting all contracted words
    contractions_keys = '|'.join(contraction_map.keys())
    contractions_pattern = re.compile(f'({contractions_keys})', flags=re.DOTALL)

    def expand_match(contraction):
        # Getting entire matched sub-string
        match = contraction.group(0)
        expanded_contraction = contraction_map.get(match)
        if not expand_contractions:
            print(match)
            return match
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


expand_contractions("y'all can't expand contractions i'd think")

'you all cannot expand contractions id think'

In [14]:
# Converting to lowercase
train_data.highlights = train_data.highlights.apply(str.lower)
train_data.article = train_data.article.apply(str.lower)

train_data.sample(5)

,id,article,highlights
52829,95d3a2c35a81b209657a9ca434c0d0706a79db83,disgraced: dave lee travis leaves southwark cr...,alleged victim's case was not pursued by publi...
115751,2162085e4ea2d7fbf718bd6dc4004729a75d4c9f,it's the picture we needed to see after the pa...,"sgt. bret barnum says he approached boy ""not a..."
181131,76778a41c7fec62c22e345a072f955f3ccfc32ed,by . reuters reporter . a classified u.s. sena...,"two pages of a 6,600- word torture document ob..."
188874,80a02438636cd9bd6dd6f7f5ded1e68aafc41534,"princeton, new jersey (cnn) -- a new restored ...",charlie chaplin's 1930s film depicted anxiety ...
137525,3ddafe49681538302a637789c51a646a1f41de04,by . daily mail reporter . published: . 08:02 ...,"darren newton convicted after stealing £6,500-..."


In [15]:
# Converting to lowercase
test_data.highlights = test_data.highlights.apply(str.lower)
test_data.article = test_data.article.apply(str.lower)

test_data.sample(5)

,id,article,highlights
8604,a85f083696278a765ae387e202ec1c1edb56e6ea,the attorney for an arizona city where 19 fire...,prescott city attorney jon paladini claims sol...
4425,7b68be37b256d43a8a5008c519506bdd7141f4e7,the man whose father married tippi hedren and ...,"roar, the film that took 11 years to create an..."
8351,433a46f34fae3568d413c3d4dda48fe72887bc41,fa cup semi-finalists reading still have linge...,conor mcaleny scores stoppage time equaliser f...
6009,b24de20738cc9e810500fd8f1a516cadf15397b2,two black candidates were among three people e...,"ferguson, missouri elected three black officia..."
4403,6de323acb06b2590f5aba38ae450865ea4e793b1,(cnn)there's a stampede of investigators in ca...,san bernardino sheriff says 10 deputies have b...


In [16]:
# Converting to lowercase
val_data.highlights = val_data.highlights.apply(str.lower)
val_data.article = val_data.article.apply(str.lower)

val_data.sample(5)

,id,article,highlights
1250,824a8cf242d08c33257804e0136a562738bbc2af,a washington police dog has died after he inge...,labrador mix barney ingested meth during bust ...
1790,bce4ee75b4f082a80c463d9dd001065fc6c51ea2,"the message, as neil jenkins relaxed at his al...",wales are one of three teams in with a chance ...
1999,97c80fc1eca236bb19a8c7a20963067f06b36000,a real-life paddington bear who suffered such ...,cholita was in a ‘deplorable condition’ when s...
3950,f4f1537ff363e6ed381606d981324036f2bfe7aa,twitter is reportedly looking at ways to make ...,twitter is said to be looking at ways to make ...
509,5963adaa6d452d2f78d1e6329160a21bb10e3e42,there wasn't much chance of any of the pooches...,hundreds of dogs have arrived at the nec in bi...


In [17]:
#expanding contractions
train_data.highlights = train_data.highlights.apply(expand_contractions)
train_data.article = train_data.article.apply(expand_contractions)
train_data.sample(5)

,id,article,highlights
148501,4c09f2b573eb3deaed556ec26ea3531141af7abe,"san francisco, california (cnn) -- as all peo...",craigslist ceo says site uses comprehensive pr...
115518,2111090c9cb1e48e5fa507069312e32cce8fd481,by . tim shipman and gerard couzens . last upd...,cash is being handed to buenos aires at a time...
58902,a714d264d10eec8159c7ffa5074c0ddb3f117317,(cnn) -- illinois gov. rod blagojevich appeare...,"new: gov. rod blagojevich released on $4,500 b..."
233093,b9c24fa2b1ab6571d7f8f52ab40c32f00b057c3c,by . emma glanfield for mailonline . a family ...,gloucestershire family targeted vulnerable men...
202411,920a6d9291a1854f26bc8a18e36034508649bff8,(cnn) -- ivory coast says it will not transfer...,ivory coast officials want arrest warrant agai...


In [18]:
#expanding contractions
test_data.highlights = test_data.highlights.apply(expand_contractions)
test_data.article = test_data.article.apply(expand_contractions)
test_data.sample(5)

,id,article,highlights
3339,cb91fc3c8528ad8885d81f120e60a28312e5dc1c,a brewery has seen a spike in sales following ...,bradford brewery tweeted george galloway askin...
767,b35a0c9ceceba8e01f7f3a66aa32b77082efba80,after tony blair’s speech in his former consti...,an earlier version of this article wrongly ref...
6908,86e61b2c8faf012763d0461415336dd788e3c12a,a 24-year-old san diego man . was arrested and...,"mohamad saeed kodaimati, 24, faced two counts ..."
3622,517a2943ee6fab9e6a81bd5cd259e5801682d190,aston villa manager tim sherwood kept faith wi...,jack grealish was excellent in fa cup semi fin...
4256,47681f33b383d3000a13cff5c5429771204619ed,barcelona boss luis enrique has urged his play...,luis enrique insists his side will be focusing...


In [19]:
#expanding contractions
val_data.highlights = val_data.highlights.apply(expand_contractions)
val_data.article = val_data.article.apply(expand_contractions)
val_data.sample(5)

,id,article,highlights
11281,091a26280a36d36926e066a6018e2941ff87d5dd,"swaggering away from court: lee cairns, 43, an...",lee cairns took running leap at stan roberts a...
12909,c24e1282c1d0fb4a79a3d2ceee58d53a8bdfb38f,teddy sheringham says james wilson could score...,teddy sheringham has lavished praise on man un...
8161,5605969819cd5a1d21a6cb48955895b90b87d6f8,a texas man who is finally no longer required ...,"willie carson of houston, texas has had his wa..."
11643,d537680f5809e6675c02f85108332daa163e06a5,tim sherwood has revealed charles nzogbias ren...,tim sherwood wants charles nzogbia to rediscov...
11492,f8b1272935c06d2f3d68918208ef7add64246cdf,"(cnn)cabelas not a cat, but the young pit bull...",police found the pup after neighbors heard gun...


In [20]:
# Remove puncuation from word
def rm_punc_from_word(word):
    clean_alphabet_list = [
        alphabet for alphabet in word if alphabet not in string.punctuation
    ]
    return ''.join(clean_alphabet_list)

print(rm_punc_from_word('#cool!'))


# Remove puncuation from text
def rm_punc_from_text(text):
    clean_word_list = [rm_punc_from_word(word) for word in text]
    return ''.join(clean_word_list)

print(rm_punc_from_text("Frankly, my dear, I don't give a damn"))

cool
Frankly my dear I dont give a damn


In [21]:
# Remove numbers from text
def rm_number_from_text(text):
    text = re.sub('[0-9]+', '', text)
    return ' '.join(text.split())  # to rm `extra` white space

print(rm_number_from_text('You are 100times more sexier than me'))
print(rm_number_from_text('If you taught yes then you are 10 times more delusional than me'))

You are times more sexier than me
If you taught yes then you are times more delusional than me


In [22]:
import nltk

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
# Remove stopwords from text
def rm_stopwords_from_text(text):
    _stopwords = stopwords.words('english')
    text = text.split()
    word_list = [word for word in text if word not in _stopwords]
    return ' '.join(word_list)

rm_stopwords_from_text("Love means never having to say you're sorry")



'Love means never say sorry'

In [25]:
# Cleaning text
def clean_text(text):
    text = text.lower()
    text = rm_punc_from_text(text)
    text = rm_number_from_text(text)
    text = rm_stopwords_from_text(text)

    # there are hyphen(–) in many titles, so replacing it with empty str
    # this hyphen(–) is different from normal hyphen(-)
    text = re.sub('–', '', text)
    text = ' '.join(text.split())  # removing `extra` white spaces

    # Removing unnecessary characters from text
    text = re.sub("(\\t)", ' ', str(text)).lower()
    text = re.sub("(\\r)", ' ', str(text)).lower()
    text = re.sub("(\\n)", ' ', str(text)).lower()

    # remove accented chars ('Sómě Áccěntěd těxt' => 'Some Accented text')
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode(
        'utf-8', 'ignore'
    )

    text = re.sub("(__+)", ' ', str(text)).lower()
    text = re.sub("(--+)", ' ', str(text)).lower()
    text = re.sub("(~~+)", ' ', str(text)).lower()
    text = re.sub("(\+\++)", ' ', str(text)).lower()
    text = re.sub("(\.\.+)", ' ', str(text)).lower()

    text = re.sub(r"[<>()|&©ø\[\]\'\",;?~*!]", ' ', str(text)).lower()

    text = re.sub("(mailto:)", ' ', str(text)).lower()
    text = re.sub(r"(\\x9\d)", ' ', str(text)).lower()
    text = re.sub("([iI][nN][cC]\d+)", 'INC_NUM', str(text)).lower()
    text = re.sub("([cC][mM]\d+)|([cC][hH][gG]\d+)", 'CM_NUM',
                  str(text)).lower()

    text = re.sub("(\.\s+)", ' ', str(text)).lower()
    text = re.sub("(\-\s+)", ' ', str(text)).lower()
    text = re.sub("(\:\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    try:
        url = re.search(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', str(text))
        repl_url = url.group(3)
        text = re.sub(r'((https*:\/*)([^\/\s]+))(.[^\s]+)', repl_url, str(text))
    except Exception as e:
        pass

    text = re.sub("(\s+)", ' ', str(text)).lower()
    text = re.sub("(\s+.\s+)", ' ', str(text)).lower()

    return text

clean_text("Mrs. Robinson, you're trying to frighten, aren't you?")

'mrs robinson youre trying frighten arent'

In [26]:
train_data.article = train_data.article.apply(clean_text)
train_data.highlights = train_data.highlights.apply(clean_text)
train_data.sample(5)

,id,article,highlights
193032,85ed282a8fb9ae37002b2811d845f0ac77861212,man behind grotesque animal slter porn site sh...,adam redfords mother dter inventor hammond ele...
202796,9292a43e89d9de6c29d14323c858386bfcab679e,man whose new york city murder conviction over...,anthony yarbough whose new york city murder co...
236075,bd9bb33aef5ef0e14c4191ecfee97b163d3495b3,stephen mcgowan daily mail peter lawwell urged...,bhoys chief executive peter lawwell called clu...
74321,d2bf3f2a3b8086411b01141147a4eb4ce402f32a,daily mail reporter published est ber updated ...,mya barry months old died ember opiate poisoni...
250823,d09cb41986b465ef1aef567a51dfb5f715a24260,wealthy chinese businessman jailed years hirin...,wealthy chinese businessman xi pat created net...
